In [1]:
import numpy as np
import cv2

In [ ]:
## haar cascade (고양이 얼굴 검출기)

cat_face_cascade = cv2.CascadeClassifier('haarcascade_frontalcatface_extended.xml')

img = cv2.imread('image/cats.jpg', 1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cat_faces = cat_face_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in cat_faces :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## haar cascade (눈, 얼굴 검출기)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

img = cv2.imread('image/aespa.jpg', 1)
img = cv2.resize(img, dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 3)
eyes = eye_cascade.detectMultiScale(gray, 1.1, 3)


for (x,y,w,h) in faces :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    
    
for (x,y,w,h) in eyes :
    
    cv2.rectangle(img,(x,y), (x+w,y+h), (255,0,0), 2)
    
    
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## ORB 알고리즘(Oriented fast and Rotated Breif)

src = cv2.imread('image/apple_book.jpg', 1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

target = cv2.imread('image/apple.jpg', 0)

orb = cv2.ORB_create(
    nfeatures = 40000,
    scaleFactor = 1.2,
    nlevels = 8,
    edgeThreshold = 31,
    firstLevel = 0,
    WTA_K = 2,
    scoreType = cv2.ORB_HARRIS_SCORE,
    patchSize = 31,
    fastThreshold = 20,
    
)

kp1, des1 = orb.detectAndCompute(gray, None)
kp2, des2 = orb.detectAndCompute(target, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x: x.distance)

for i in matches[:100]:
    idx = i.queryIdx
    x1, y1, =  kp1[idx].pt
    cv2.circle(src, (int(x1), int(y1)), 3, (255, 0, 0), 3)
    
cv2.imshow('src', src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## K - means clustering (평균 군집화) // 비지도 학습

src = cv2.imread('image/flower.jpg', 1)

data = src.reshape(-1,3).astype(np.float32)
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)
retval, bestLabels, centers = cv2.kmeans(data, 5, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = centers.astype(np.uint8)
dst = centers[bestLabels].reshape(src.shape)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
## K - nearest neighbor (최근접 이웃) // 지도학습

def loadTrainData(image_path, label_path):
    
    with open(image_path, "rb") as image_data:
        images = np.frombuffer(image_data.read(), dtype = np.uint8, offset = 16)
        
    with open(label_path, "rb") as label_data:
        labels = np.frombuffer(label_data.read(), dtype = np.uint8, offset = 8)
        
    return images.reshape(-1, 784), labels


train_x, train_y = loadTrainData(
    
    "./fashion-mnist/train-images-idx3-ubyte",
    "./fashion-mnist/train-labels-idx1-ubyte"
)

test_x, test_y = loadTrainData(
    
    "./fashion-mnist/t10k-images-idx3-ubyte",
    "./fashion-mnist/t10k-labels-idx1-ubyte"
)


label_dict = {
    
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot",
    
}

knn = cv2.ml.KNearest_create()
retval = knn.train(train_x.astype(np.float32), cv2.ml.ROW_SAMPLE, train_y.astype(np.int32))

count = 500

retval, results, neighborResponses, dist = knn.findNearest(
    test_x[:count].astype(np.float32), k = 7
    
)

matches = results.astype(np.uint8) == test_y[:count][:, None]

print(np.count_nonzero(matches) / count * 100)


for idx, result in enumerate(results):
    print("Index : {}".format(idx))
    print("예측값 : {}".format(label_dict[int(result)]))
    print("싧제값 : {}".format(label_dict[test_y[idx]]))
    cv2.imshow("images", test_x[idx].reshape(28,28,1))
    cv2.waitKey()

In [8]:
## optical flow (루카스 - 카나데 알고리즘)

cap = cv2.VideoCapture('image/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

color = (0,255,0)
lines = None
prevImg = None
## calcOpticalFlowPyrLK 중지 요견 설정
termcriteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    img_draw = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prevImg is None:
        prevImg = gray
        
        lines = np.zeros_like(frame)
        
        prevPt = cv2.goodFeaturesToTrack(prevImg, 200, 0.01, 10)
        
    else:
        nextImg = gray
        
        nextPt, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, \
                                                      prevPt, None, \
                                                      criteria=termcriteria)
        
        prevMv = prevPt[status==1]
        nextMv = nextPt[status==1]
        for i,(p, n) in enumerate(zip(prevMv, nextMv)):
            px,py = p.ravel()
            nx,ny = n.ravel()
            
            cv2.line(lines, (px.astype(np.uint16),py.astype(np.uint16)), \
                     (nx.astype(np.uint16),ny.astype(np.uint16)), \
                     (0,255,0), 2)
            
            cv2.circle(img_draw, (nx.astype(np.uint16),ny.astype(np.uint16)),\
                       2, (0,255,0), -1)
            
        img_draw = cv2.add(img_draw,lines)
        
        prevImg = nextImg
        prevPt = nextMv.reshape(-1,1,2)
    
    cv2.imshow('OpticalFlow_LK', img_draw)
    
    if cv2.waitKey(delay) == ord('q'):
        break
    elif cv2.waitKey(delay) == ord('r'):
        prevImg = None

cv2.destroyAllWindows()
cap.release()

In [ ]:
print(prevPt[status==1])
print(prevPt)

In [7]:
## optical flow (군나르 파너백 알고리즘)

def drawFlow(img, flow, step = 16):
    h,w = img.shape[:2]
    
    idx_y, idx_x = np.mgrid[step/2:h:step,step/2:w:step].astype(np.int)
    indices = np.stack((idx_x, idx_y), axis = -1).reshape(-1,2)
    
    for x,y in indices:
        
        cv2.circle(img, (x,y), 1, (0,255,0), -1)
        
        dx,dy = flow[y, x].astype(np.int)
        
        cv2.line(img, (x,y), (x+dx, y+dy), (0,255,0),2,cv2.LINE_AA)
        
prev = None

cap = cv2.VideoCapture('image/walking.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)

while cap.isOpened():
    ret,frame = cap.read()
    if not ret: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev is None:
        prev = gray
    else :
        flow = cv2.calcOpticalFlowFarneback(prev,gray,None,\
                                           0.5,3,15,3,5,1.1,\
                                            cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        drawFlow(frame,flow)
        
        prev = gray
        
    cv2.imshow('op', frame)
    if cv2.waitKey(delay) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

C:\Users\alsgu\AppData\Local\Temp/ipykernel_10860/3754260517.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  idx_y, idx_x = np.mgrid[step/2:h:step,step/2:w:step].astype(np.int)
C:\Users\alsgu\AppData\Local\Temp/ipykernel_10860/3754260517.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

In [17]:
## 가우시안 블러

img = cv2.imread('image/tomato.jpg', 1)
img = cv2.resize(img, dsize=(0,0), fx = 0.5, fy = 0.5,
                 interpolation = cv2.INTER_AREA)

dst = img.copy()

dst = cv2.GaussianBlur(dst, (0,0), sigmaX = 10, sigmaY = 2)

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()